In [1]:
from dask.distributed import Client

client = Client("tls://192.168.197.199:8786")
client

/usr/local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


<Client: 'tls://192.168.197.199:8786' processes=21 threads=21, memory=60.63 GiB>

In [2]:
import os
import numpy as np
import awkward as ak
import correctionlib

import dask
import copy
import hist.dask as hda
import dask_awkward as dak

from coffea import processor
from coffea.nanoevents.methods import vector
from coffea.dataset_tools import (
    apply_to_fileset,
    max_chunks,
    preprocess,
)

from coffea.nanoevents import NanoEventsFactory, BaseSchema

/tmp/ipykernel_4442/3618735414.py:12: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [3]:
class Processor(processor.ProcessorABC):
    def __init__(self, NTUPLE=False):
        self.NTUPLE = NTUPLE
        pass

    def getPtEtaPhiM(self, events, obj_name):
        if "MET" in obj_name:
            vars = ["pt", "phi"]
        else:
            vars = ["pt", "eta", "phi", "mass"]
        return {var: events[f"{obj_name}_{var}"] for var in vars}

    def getObjects(self, events, obj_name, adds=[]):
        out = self.getPtEtaPhiM(events, obj_name)
        if adds:
            out.update({add: events[f"{obj_name}_{add}"] for add in adds})
        reqs = {"pt", "eta", "phi", "mass"}
        is_4d_mom = reqs.issubset(out.keys())
        return ak.zip(out, with_name="PtEtaPhiMLorentzVector", behavior=vector.behavior) if is_4d_mom else ak.zip(out, with_name="TwoVector", behavior=vector.behavior)

    def getFatJets(self, events):
        return self.getObjects(events, "FatJet", adds=["jetId", "msoftdrop", "particleNet_XqqVsQCD"])
        
    def process(self, events):
        #mask_hlt = events.HLT_AK8DiPFJet250_250_SoftDropMass40 | events.HLT_AK8DiPFJet260_260_SoftDropMass30
        jets = self.getFatJets(events)
        mask_jets = (jets.jetId > 0) & (np.abs(jets.eta) < 2.1) & (jets.pt > 320) & (jets.msoftdrop > 60) & (jets.particleNet_XqqVsQCD > 0.65)
        jets = jets[mask_jets]
        mask_event = (ak.num(jets, axis=1) > 1)
        jets = jets[mask_event]
        jet0 = jets[:,0]
        jet1 = jets[:,1]
        dphi = jet0.deltaphi(jet1) > 1.6
        jets = jets[dphi]
        
        return {"jets" : 
                {
                "msoftdrop" : jets.msoftdrop[:,0],
                }
               }
    def postprocess(self, accumulator):
        pass

In [4]:
import json
#input_path = "/home/cms-jovyan/analysis/offshellW/"

def read_json(input_file):
    out = {}
    out[input_file] = {}
    full_path = f"./{input_file}.json"
    with open(full_path) as json_file:
        data = json.load(json_file)
    for key, value in data.items():
        files = {}
        for file in value["files"]:
            file = "root://xcache//store/" + file.split("/store/")[1]
            files[file] = "Events"
            #break # FIXME
        if "files" in out[input_file].keys():
            out[input_file]["files"].update(files)
        else:
            out[input_file]["files"] = files
            
        out[input_file]["metadata"] = value["metadata"]
    return out

datasets_to_process = [
    "JetMET",
]

datasets = []
for dataset in datasets_to_process:
    datasets.append(read_json(dataset))

In [6]:
outs = []
for dataset in datasets:
    dataset_runnable, dataset_updated = preprocess(
        dataset,
        align_clusters=False,
        step_size=500_000,
        files_per_batch=1,
        skip_bad_files=True,
        save_form=False,
    )
    to_compute = apply_to_fileset(
        Processor(True),
        max_chunks(dataset_runnable, 10000),
        schemaclass=BaseSchema,
    )
    (out,) = dask.compute(to_compute)
    outs.append(out)

2024-10-27 18:39:25,571 - distributed.client - WARNING - Couldn't gather 432 keys, rescheduling (('getitem-04b26b679e8892474d56a4cee4cc9fe8', 106), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 115), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 353), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 362), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 133), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 371), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 380), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 618), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 389), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 627), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 636), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 218), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 654), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 227), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 245), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 483), ('getitem-04b26b679e8892474d56a4cee4cc9fe8', 492), ('getitem-04b26b679e8892474d56a4cee4

OSError: File did not vector_read properly: [ERROR] Operation expired

In [ ]:
outs